# Inbit Weaning using 3DCNN video

### SetUp

in this section using BAIR dataset. \
But bair training dataset size is 30GB. \
now, you have downloaded 190MB test data.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow_datasets.core import SplitGenerator
from tensorflow_datasets.video.bair_robot_pushing import BairRobotPushingSmall
import tempfile
import pathlib

TEST_DIR = pathlib.Path(tempfile.mkdtemp()) / "bair_robot_pushing_small/softmotion30_44k/test/"

print(TEST_DIR)


In [ ]:
import subprocess

cmd = "mkdir -p {}".format(TEST_DIR)
# Download the test split to $TEST_DIR
subprocess.run(cmd, shell=True)
# subprocess.check_call("curl -nv https://storage.googleapis.com/download.tensorflow.org/data/bair_test_traj_0_to_255.tfrecords -O {}/traj_0_to_255.tfrecords".format(TEST_DIR), shell=True)

In [ ]:
# Since the dataset builder expects the train and test split to be downloaded,
# patch it so it only expects the test data to be available
builder = BairRobotPushingSmall()
test_generator = SplitGenerator(name='test', gen_kwargs={"filedir": str(TEST_DIR)})
builder._split_generators = lambda _: [test_generator]
builder.download_and_prepare()


### BAIR: input Numpy array

In [ ]:
# load some example data (BAIR)
batch_size = 16

# If unable to download the dataset automatically due to "not enough disk space", please download manually to Google Drive and
# load using tf.data.TFRecordDataset
ds = builder.as_dataset(split="test")
test_videos = ds.batch(batch_size)
first_batch = next(iter(test_videos))
input_frames = first_batch['image_aux1'][:, ::15]
input_frames = tf.cast(input_frames, tf.float32)

In [ ]:
# title Visualize loaded videos start and end frames

print("Test videos shape [batch_size, start/end frame, height, width, num_channels]:", input_frames.shape)
sns.set_style('white')
plt.figure(figsize=(4, 2*batch_size))

for i in range(batch_size)[:4]:
    plt.subplot(batch_size, 2, 1 + 2*i)
    plt.imshow(input_frames[i, 0] / 255.0)
    plt.title(f'Video {i}: First frame'.format(i))
    plt.axis('off')
    plt.subplot(batch_size, 2, 2 + 2*i)
    plt.imshow(input_frames[i, 1] / 255.0)
    plt.title(f"Video {i}: Last frame")
    plt.axis('off')

In [ ]:
# Loading Hub module
hub_handle = 'https://tfhub.dev/google/tweening_conv3d_bair/1'
module = hub.load(hub_handle).signatures['default']

### Generate and visualize video

In [ ]:
filled_frames = module(input_frames)['default'] / 255.0

In [ ]:
# Show sequences of generated video frames
# Concatenate start/end frames and the generated filled frames for the new videos

generated_videos = np.concatenate([input_frames[:, :1] / 255.0, filled_frames, input_frames[:, 1:] / 255.0], axis=1)

for video_id in range(4):
    fig = plt.figure(figsize=(10*2, 2))
    for frame_id in range(1, 16):
        ax = fig.add_axes([frame_id * 1 / 16., 0, (frame_id + 1) * 1 / 16., 1], xmargin=0, ymargin=0)
        ax.imshow(generated_videos[video_id, frame_id])
        ax.axis('off')